# Building Our Chatbot Framework

We'll build a simple state-machine to handle responses, using our intents model (from the previous step) as our classifier. That's how chatbots work. After loading the same imports, we'll un-pickle our model and documents as well as reload our intents file. Remember our chatbot framework is separate from our model build - you don't need to rebuild your model unless the intent pattern changes. With several hundred intents and thousands of patterns the model could take several minutes to build.

In [1]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
# restore all of our data structures
import pickle
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import our chat-bot intents file
import json
with open('intents1.json') as json_data:
    intents = json.load(json_data)

In [3]:
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

Before we can begin processing intents, we need a way to produce a bag-of-words from *user input*. This is the same technique as we used earlier to create our training documents.

In [ ]:
# this is now for new data without labels! Bad: duplicate code --> refactor

def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence]
    return sentence_words

# return bag of words array: 0 or 1 for each word i the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words